<a href="https://colab.research.google.com/github/markumreed/colab_sklearn/blob/main/covid_and_financial_markets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Time Series Data
1. Covid 19 Dataset: John's Hopkins
1. Financial/Stock Data


## Covid 19 Datasets

[John's Hopkins Data](https://github.com/CSSEGISandData/COVID-19) can be found here. 

```shell
git clone https://github.com/CSSEGISandData/COVID-19.git
```

In [1]:
!git clone https://github.com/CSSEGISandData/COVID-19.git

Cloning into 'COVID-19'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 104397 (delta 0), reused 2 (delta 0), pack-reused 104391
Receiving objects: 100% (104397/104397), 877.56 MiB | 28.90 MiB/s, done.
Resolving deltas: 100% (74021/74021), done.
Checking out files: 100% (970/970), done.


In [5]:
%%shell
ls COVID-19/archived_data/archived_time_series/

README.md
time_series_19-covid-Confirmed_archived_0325.csv
time_series_19-covid-Deaths_archived_0325.csv
time_series_19-covid-Recovered_archived_0325.csv
time_series_2019-ncov-Confirmed.csv
time_series_2019-ncov-Deaths.csv
time_series_2019-ncov-Recovered.csv


## Preamble

In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
%matplotlib inline

In [7]:
path = "COVID-19/archived_data/archived_time_series/"
confirmed = pd.read_csv(path + "time_series_2019-ncov-Confirmed.csv")
deaths = pd.read_csv(path + "time_series_2019-ncov-Deaths.csv")
recovered = pd.read_csv(path + "time_series_2019-ncov-Recovered.csv")

In [10]:
confirmed.head(2)

,Province/State,Country/Region,Lat,Long,1/21/20 22:00,1/22/20 12:00,1/23/20 12:00,1/24/20 0:00,1/24/20 12:00,1/25/20 0:00,1/25/20 12:00,1/25/20 22:00,1/26/20 11:00,1/26/20 23:00,1/27/20 9:00,1/27/20 19:00,1/27/20 20:30,1/28/20 13:00,1/28/20 18:00,1/28/20 23:00,1/29/20 13:30,1/29/20 14:30,1/29/20 21:00,1/30/20 11:00,1/31/20 14:00,2/1/20 10:00,2/2/20 21:00,2/3/20 21:00,2/4/20 9:40,2/4/20 22:00,2/5/20 9:00,2/5/20 23:00,2/6/20 9:00,2/6/20 14:20,2/7/20 20:13,2/7/20 22:50,2/8/20 10:24,2/8/20 23:04,2/9/20 10:30,2/9/20 23:20,2/10/20 10:30,2/10/20 19:30,2/11/20 10:50,2/11/20 20:44,2/12/20 10:20,2/12/20 22:00,2/13/20 10:00,2/13/20 21:15,2/14/20 11:23
0,Anhui,Mainland China,31.82571,117.2264,NaN,1.0,9.0,15.0,15.0,39.0,39.0,60.0,60.0,70.0,70.0,70.0,106.0,106.0,106.0,152.0,152.0,152.0,200.0,200.0,237.0,297.0,408.0,480.0,480.0,530.0,530.0,591.0,591.0,591.0,665.0,733.0,733.0,779.0,779.0,830.0,830.0,830.0,860.0,889.0,889.0,910.0,910.0,934,934
1,Beijing,Mainland China,40.18238,116.4142,10.0,14.0,22.0,26.0,36.0,36.0,41.0,51.0,68.0,68.0,72.0,80.0,80.0,91.0,91.0,91.0,111.0,111.0,111.0,114.0,139.0,168.0,191.0,212.0,212.0,228.0,253.0,274.0,274.0,274.0,297.0,315.0,315.0,326.0,326.0,337.0,337.0,337.0,342.0,342.0,352.0,366.0,366.0,366,372


In [12]:
deaths.head(2)

,Province/State,Country/Region,Lat,Long,1/21/20 22:00,1/22/20 12:00,1/23/20 12:00,1/24/20 0:00,1/24/20 12:00,1/25/20 0:00,1/25/20 12:00,1/25/20 22:00,1/26/20 11:00,1/26/20 23:00,1/27/20 9:00,1/27/20 19:00,1/27/20 20:30,1/28/20 13:00,1/28/20 18:00,1/28/20 23:00,1/29/20 13:30,1/29/20 14:30,1/29/20 21:00,1/30/20 11:00,1/31/20 14:00,2/1/20 10:00,2/2/20 21:00,2/3/20 21:00,2/4/20 9:40,2/4/20 22:00,2/5/20 9:00,2/5/20 23:00,2/6/20 9:00,2/6/20 14:20,2/7/20 20:13,2/7/20 22:50,2/8/20 10:24,2/8/20 23:04,2/9/20 10:30,2/9/20 23:20,2/10/20 10:30,2/10/20 19:30,2/11/20 10:50,2/11/20 20:44,2/12/20 10:20,2/12/20 22:00,2/13/20 10:00,2/13/20 21:15,2/14/20 11:23
0,Anhui,Mainland China,31.82571,117.2264,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,1.0,3.0,3.0,3.0,4.0,4.0,4.0,5.0,5.0,6,6
1,Beijing,Mainland China,40.18238,116.4142,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,3.0,3.0,3.0,3.0,3.0,3,3


In [13]:
recovered.head(2)

,Province/State,Country/Region,Lat,Long,1/21/20 22:00,1/22/20 12:00,1/23/20 12:00,1/24/20 0:00,1/24/20 12:00,1/25/20 0:00,1/25/20 12:00,1/25/20 22:00,1/26/20 11:00,1/26/20 23:00,1/27/20 9:00,1/27/20 19:00,1/27/20 20:30,1/28/20 13:00,1/28/20 18:00,1/28/20 23:00,1/29/20 13:30,1/29/20 14:30,1/29/20 21:00,1/30/20 11:00,1/31/20 14:00,2/1/20 10:00,2/2/20 21:00,2/3/20 21:00,2/4/20 9:40,2/4/20 22:00,2/5/20 9:00,2/5/20 23:00,2/6/20 9:00,2/6/20 14:20,2/7/20 20:13,2/7/20 22:50,2/8/20 10:24,2/8/20 23:04,2/9/20 10:30,2/9/20 23:20,2/10/20 10:30,2/10/20 19:30,2/11/20 10:50,2/11/20 20:44,2/12/20 10:20,2/12/20 22:00,2/13/20 10:00,2/13/20 21:15,2/14/20 11:23
0,Anhui,Mainland China,31.82571,117.2264,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,2.0,2.0,3.0,5.0,7.0,14.0,14.0,20.0,23.0,23.0,34.0,34.0,47.0,47.0,59.0,59.0,72.0,73.0,88.0,88.0,105.0,108.0,127.0,128.0,157.0,166,193
1,Beijing,Mainland China,40.18238,116.4142,NaN,NaN,NaN,NaN,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,5.0,9.0,9.0,12.0,12.0,23.0,24.0,31.0,31.0,31.0,33.0,34.0,34.0,37.0,37.0,44.0,44.0,44.0,48.0,48.0,56.0,68.0,69.0,69,80


## Reshape by Melting

In [14]:
confirmed.columns

Index(['Province/State', 'Country/Region', 'Lat', 'Long', '1/21/20 22:00',
       '1/22/20 12:00', '1/23/20 12:00', '1/24/20 0:00', '1/24/20 12:00',
       '1/25/20 0:00', '1/25/20 12:00', '1/25/20 22:00', '1/26/20 11:00',
       '1/26/20 23:00', '1/27/20 9:00', '1/27/20 19:00', '1/27/20 20:30',
       '1/28/20 13:00', '1/28/20 18:00', '1/28/20 23:00', '1/29/20 13:30',
       '1/29/20 14:30', '1/29/20 21:00', '1/30/20 11:00', '1/31/20 14:00',
       '2/1/20 10:00', '2/2/20 21:00', '2/3/20 21:00', '2/4/20 9:40',
       '2/4/20 22:00', '2/5/20 9:00', '2/5/20 23:00', '2/6/20 9:00',
       '2/6/20 14:20', '2/7/20 20:13', '2/7/20 22:50', '2/8/20 10:24',
       '2/8/20 23:04', '2/9/20 10:30', '2/9/20 23:20', '2/10/20 10:30',
       '2/10/20 19:30', '2/11/20 10:50', '2/11/20 20:44', '2/12/20 10:20',
       '2/12/20 22:00', '2/13/20 10:00', '2/13/20 21:15', '2/14/20 11:23'],
      dtype='object')

In [15]:
confirmed.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], var_name="date", value_name="confirmed")

,Province/State,Country/Region,Lat,Long,date,confirmed
0,Anhui,Mainland China,31.82571,117.2264,1/21/20 22:00,NaN
1,Beijing,Mainland China,40.18238,116.4142,1/21/20 22:00,10.0
2,Chongqing,Mainland China,30.05718,107.8740,1/21/20 22:00,5.0
3,Fujian,Mainland China,26.07783,117.9895,1/21/20 22:00,NaN
4,Gansu,Mainland China,36.06110,103.8343,1/21/20 22:00,NaN
...,...,...,...,...,...,...
3325,NaN,Belgium,50.50390,4.4699,2/14/20 11:23,1.0
3326,"Madison, WI",US,43.07310,-89.4012,2/14/20 11:23,1.0
3327,Diamond Princess cruise ship,Others,35.44370,129.6380,2/14/20 11:23,218.0
3328,"San Diego County, CA",US,32.71570,-117.1611,2/14/20 11:23,2.0


In [18]:
def covid_melt(df, value_name=None):
  return df.melt(id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'], var_name="date", value_name=value_name)

In [22]:
confirmed_melt = covid_melt(confirmed, "confirmed")
deaths_melt = covid_melt(deaths, "deaths")
recovered_melt = covid_melt(recovered, "recovered")

In [27]:
covid_data = confirmed_melt.merge(deaths_melt).merge(recovered_melt)

In [29]:
covid_data.head(2)

,Province/State,Country/Region,Lat,Long,date,confirmed,deaths,recovered
0,Anhui,Mainland China,31.82571,117.2264,1/21/20 22:00,NaN,NaN,NaN
1,Beijing,Mainland China,40.18238,116.4142,1/21/20 22:00,10.0,NaN,NaN


In [30]:
covid_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3330 entries, 0 to 3329
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Province/State  2385 non-null   object 
 1   Country/Region  3330 non-null   object 
 2   Lat             3330 non-null   float64
 3   Long            3330 non-null   float64
 4   date            3330 non-null   object 
 5   confirmed       2706 non-null   float64
 6   deaths          1184 non-null   float64
 7   recovered       1508 non-null   float64
dtypes: float64(5), object(3)
memory usage: 234.1+ KB


## Date to DateTime Object

In [33]:
covid_data['date'] = pd.to_datetime(covid_data['date'])

In [34]:
covid_data.head()

,Province/State,Country/Region,Lat,Long,date,confirmed,deaths,recovered
0,Anhui,Mainland China,31.82571,117.2264,2020-01-21 22:00:00,NaN,NaN,NaN
1,Beijing,Mainland China,40.18238,116.4142,2020-01-21 22:00:00,10.0,NaN,NaN
2,Chongqing,Mainland China,30.05718,107.8740,2020-01-21 22:00:00,5.0,NaN,NaN
3,Fujian,Mainland China,26.07783,117.9895,2020-01-21 22:00:00,NaN,NaN,NaN
4,Gansu,Mainland China,36.06110,103.8343,2020-01-21 22:00:00,NaN,NaN,NaN


In [35]:
covid_data.set_index('date', inplace=True)

In [36]:
covid_data.head()

,Province/State,Country/Region,Lat,Long,confirmed,deaths,recovered
date,,,,,,,
2020-01-21 22:00:00,Anhui,Mainland China,31.82571,117.2264,NaN,NaN,NaN
2020-01-21 22:00:00,Beijing,Mainland China,40.18238,116.4142,10.0,NaN,NaN
2020-01-21 22:00:00,Chongqing,Mainland China,30.05718,107.8740,5.0,NaN,NaN
2020-01-21 22:00:00,Fujian,Mainland China,26.07783,117.9895,NaN,NaN,NaN
2020-01-21 22:00:00,Gansu,Mainland China,36.06110,103.8343,NaN,NaN,NaN


In [40]:
covid_data.columns = [c.lower().replace("/","_") for c in covid_data.columns]

In [44]:
covid_data.head()

,province_state,country_region,lat,long,confirmed,deaths,recovered
date,,,,,,,
2020-01-21 22:00:00,Anhui,Mainland China,31.82571,117.2264,NaN,NaN,NaN
2020-01-21 22:00:00,Beijing,Mainland China,40.18238,116.4142,10.0,NaN,NaN
2020-01-21 22:00:00,Chongqing,Mainland China,30.05718,107.8740,5.0,NaN,NaN
2020-01-21 22:00:00,Fujian,Mainland China,26.07783,117.9895,NaN,NaN,NaN
2020-01-21 22:00:00,Gansu,Mainland China,36.06110,103.8343,NaN,NaN,NaN


## Filling NA values

1. confirmed
1. deaths
1. recovered

In [45]:
fill_values = {"confirmed":0, "recovered":0, "deaths":0} # fill values dictionary; helps control which variables are filled

For the variables confirmed, recovered, and deaths we fill the NaN with zeros. For all other variables, we left the missing values as NaN.

In [46]:
covid_data.fillna(value=fill_values)

,province_state,country_region,lat,long,confirmed,deaths,recovered
date,,,,,,,
2020-01-21 22:00:00,Anhui,Mainland China,31.82571,117.2264,0.0,0.0,0.0
2020-01-21 22:00:00,Beijing,Mainland China,40.18238,116.4142,10.0,0.0,0.0
2020-01-21 22:00:00,Chongqing,Mainland China,30.05718,107.8740,5.0,0.0,0.0
2020-01-21 22:00:00,Fujian,Mainland China,26.07783,117.9895,0.0,0.0,0.0
2020-01-21 22:00:00,Gansu,Mainland China,36.06110,103.8343,0.0,0.0,0.0
...,...,...,...,...,...,...,...
2020-02-14 11:23:00,NaN,Belgium,50.50390,4.4699,1.0,0.0,0.0
2020-02-14 11:23:00,"Madison, WI",US,43.07310,-89.4012,1.0,0.0,0.0
2020-02-14 11:23:00,Diamond Princess cruise ship,Others,35.44370,129.6380,218.0,0.0,0.0


In [47]:
covid_data.query("province_state == 'Beijing'")

,province_state,country_region,lat,long,confirmed,deaths,recovered
date,,,,,,,
2020-01-21 22:00:00,Beijing,Mainland China,40.18238,116.4142,10.0,NaN,NaN
2020-01-22 12:00:00,Beijing,Mainland China,40.18238,116.4142,14.0,NaN,NaN
2020-01-23 12:00:00,Beijing,Mainland China,40.18238,116.4142,22.0,NaN,NaN
2020-01-24 00:00:00,Beijing,Mainland China,40.18238,116.4142,26.0,NaN,NaN
2020-01-24 12:00:00,Beijing,Mainland China,40.18238,116.4142,36.0,NaN,1.0
2020-01-25 00:00:00,Beijing,Mainland China,40.18238,116.4142,36.0,NaN,1.0
2020-01-25 12:00:00,Beijing,Mainland China,40.18238,116.4142,41.0,NaN,2.0
2020-01-25 22:00:00,Beijing,Mainland China,40.18238,116.4142,51.0,NaN,2.0
2020-01-26 11:00:00,Beijing,Mainland China,40.18238,116.4142,68.0,NaN,2.0
